Updated: 08/02/24

workbook to test normalisation of alphas 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qnmfitsrd as qnmfits
from multiprocessing import Pool
from spatial_reconstruction_tests.spatial_reconstruction import *
from matplotlib.animation import FuncAnimation
from scipy.interpolate import UnivariateSpline
from Visualisation.qnm_vis import * 
from qnmfitsrd.CCE_file_getter import *
import datetime
from scipy.optimize import least_squares

l_max = 8
n_max = 8

In [2]:
spherical_modes = [(2,2)] 

qnms = [(l,2,0,1) for l in np.arange(2, l_max+1)]

indices_lists_l = [[lm_mode+mode for mode in qnms] for lm_mode in spherical_modes]
mu_lists = [qnm.mu_list(indices, 0.5) for indices in indices_lists_l]

sum = 0 

for mu in mu_lists:
    sum += abs(mu[0])**2 

print(sum)

0.9980192115848097


In [4]:
spherical_modes = [(4,4)] 

qnms = [(l,2,0,1,lp,2,0,1) for l in np.arange(2, l_max+1) for lp in np.arange(2,l_max+1)]

indices_lists_q = [[lm_mode+mode for mode in qnms] for lm_mode in spherical_modes]
alpha_lists = [qnm.alpha(indices, 0.5) for indices in indices_lists_q]

sum = 0 

for mu in alpha_lists:
    sum += abs(mu[0])**2

print(sum)

1.0000000000000009
